In [1]:
using DifferentialEquations, ODE, ODEInterfaceDiffEq

## Define the ThreeBody Problem
const threebody_μ = parse(Float64,"0.012277471"); const threebody_μ′ = 1 - threebody_μ

f = (t,u,du) -> begin
  # 1 = y₁
  # 2 = y₂
  # 3 = y₁'
  # 4 = y₂'
  D₁ = ((u[1]+threebody_μ)^2 + u[2]^2)^(3/2)
  D₂ = ((u[1]-threebody_μ′)^2 + u[2]^2)^(3/2)
  du[1] = u[3]
  du[2] = u[4]
  du[3] = u[1] + 2u[4] - threebody_μ′*(u[1]+threebody_μ)/D₁ - threebody_μ*(u[1]-threebody_μ′)/D₂
  du[4] = u[2] - 2u[3] - threebody_μ′*u[2]/D₁ - threebody_μ*u[2]/D₂
end


t₀ = 0.0; T = parse(Float64,"17.0652165601579625588917206249")
tspan = (t₀,2T)

prob = ODEProblem(f,[0.994, 0.0, 0.0, parse(Float64,"-2.00158510637908252240537862224")],tspan)

test_sol = TestSolution(T,[prob.u0])
abstols = 1./10.^(3:13)
reltols = 1./10.^(0:10);

using Plots; gr()

Plots.GRBackend()

See that it's periodic in the chosen timespan:

In [2]:
sol = solve(prob,abstol=1e-14,reltol=1e-14)
@show sol[1] - sol[end]
@show sol[end] - prob.u0;

sol[1] - sol[end] = [9.88596e-11, 2.92296e-10, 4.76985e-8, 1.53873e-8]
sol[end] - prob.u0 = [-9.88596e-11, -2.92296e-10, -4.76985e-8, -1.53873e-8]


In [3]:
apr = appxtrue(sol,test_sol)
@show sol[end]
@show apr.u[end]
@show apr.errors

sol[end] = [0.994, -2.92296e-10, -4.76985e-8, -2.00159]
apr.u[end] = [0.994, -2.92296e-10, -4.76985e-8, -2.00159]
apr.errors = Dict(:final=>1.58692e-8)


Dict{Symbol,Float64} with 1 entry:
  :final => 1.58692e-8

This three-body problem is known to be a tough problem. Let's see how the algorithms fair at standard tolerances.

### 5th Order Runge-Kutta Methods


In [4]:
setups = [Dict(:alg=>DP5())
          #Dict(:alg=>ode45()) #fails
          Dict(:alg=>BS5())
          Dict(:alg=>Tsit5())
          Dict(:alg=>dopri5())];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,save_everystep=false,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.BS5 
 
 
 
 OrdinaryDiffEq.Tsit5 
 
 
 
 ODEInterfaceDiffEq.dopri5

#### Full save, but no dense

In [5]:
setups = [Dict(:alg=>DP5(),:dense=>false)
          #Dict(:alg=>ode45()) # Fails
          Dict(:alg=>BS5(),:dense=>false)
          Dict(:alg=>Tsit5(),:dense=>false)
          Dict(:alg=>dopri5())];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.BS5 
 
 
 
 OrdinaryDiffEq.Tsit5 
 
 
 
 ODEInterfaceDiffEq.dopri5

#### Dense

In [6]:
setups = [Dict(:alg=>DP5())
          #Dict(:alg=>ode45()) #fails
          Dict(:alg=>BS5())
          Dict(:alg=>Tsit5())
          Dict(:alg=>dopri5())];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.BS5 
 
 
 
 OrdinaryDiffEq.Tsit5 
 
 
 
 ODEInterfaceDiffEq.dopri5

In these tests we see that most of the algorithms are close,with `BS5` and `DP5` showing much better than `Tsit5`. `ode45` errors.

### Higher Order Algorithms

In [7]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern6())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>dop853())
          Dict(:alg=>Vern9())];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,save_everystep=false,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.Vern6 
 
 
 
 OrdinaryDiffEq.Vern7 
 
 
 
 OrdinaryDiffEq.TanYam7 
 
 
 
 OrdinaryDiffEq.Vern8 
 
 
 
 OrdinaryDiffEq.DP8 
 
 
 
 ODEInterfaceDiffEq.dop853 
 
 
 
 OrdinaryDiffEq.Vern9

In [8]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern6())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>dop853())
          Dict(:alg=>Vern9())];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,dense=false,numruns=100,verbose=false)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.Vern6 
 
 
 
 OrdinaryDiffEq.Vern7 
 
 
 
 OrdinaryDiffEq.TanYam7 
 
 
 
 OrdinaryDiffEq.Vern8 
 
 
 
 OrdinaryDiffEq.DP8 
 
 
 
 ODEInterfaceDiffEq.dop853 
 
 
 
 OrdinaryDiffEq.Vern9

In [9]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern6())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>dop853())
          Dict(:alg=>Vern9())];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.Vern6 
 
 
 
 OrdinaryDiffEq.Vern7 
 
 
 
 OrdinaryDiffEq.TanYam7 
 
 
 
 OrdinaryDiffEq.Vern8 
 
 
 
 OrdinaryDiffEq.DP8 
 
 
 
 ODEInterfaceDiffEq.dop853 
 
 
 
 OrdinaryDiffEq.Vern9

In this test we see `:Vern6` and `:Vern7` shine.

### Other Algorithms

Once again we separate ODE.jl because it fails:

In [ ]:
setups = [Dict(:alg=>ode78())];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,dense=false,numruns=100)

In [11]:
setups = [Dict(:alg=>DP5())
          Dict(:alg=>Vern7())
          Dict(:alg=>TanYam7())
          Dict(:alg=>Vern8())
          Dict(:alg=>DP8())
          Dict(:alg=>odex())
          Dict(:alg=>CVODE_Adams())
    ];
wp = WorkPrecisionSet(prob,abstols,reltols,setups;appxsol=test_sol,save_everystep=false,numruns=100)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 OrdinaryDiffEq.DP5 
 
 
 
 OrdinaryDiffEq.Vern7 
 
 
 
 OrdinaryDiffEq.TanYam7 
 
 
 
 OrdinaryDiffEq.Vern8 
 
 
 
 OrdinaryDiffEq.DP8 
 
 
 
 ODEInterfaceDiffEq.odex 
 
 
 
 Sundials.CVODE 
 
 
 Adams 
 
 
 :Functional,:None

Again, on cheap function calculations the Adams methods are shown to not be efficient once the error is sufficiently small. Also, as seen in other places, the extrapolation methods do not fare as well as the Runge-Kutta methods.

### Conclusion

As in the other tests, the OrdinaryDiffEq.jl algorithms with the Verner Efficient methods are the most efficient solvers at stringent tolerances for most of the tests, while the order 5 methods do well at cruder tolerances. ODE.jl fails to run the test problems without erroring.